# PSYCHROMETRICS

## Overview
This function provides core psychrometric calculations, including wet bulb temperature, dry bulb temperature, dew point, and relative humidity. It is designed for use in engineering, HVAC, and meteorological applications.  Psychrometrics involves the study of the thermodynamic properties of moist air. The key calculations performed by the `psychrometrics` function are:

- **Wet Bulb Temperature ($T_{wb}$):** The temperature a parcel of air would have if it were cooled to saturation (100% relative humidity) by evaporation, with the latent heat supplied by the parcel. It is found by solving the energy balance equation for moist air.
- **Dew Point ($T_{dp}$):** The temperature at which air becomes saturated with moisture and water vapor begins to condense. It is calculated using the Magnus formula:
  
  $T_{dp} = \frac{T_n \ln\left(\frac{p_w}{A}\right)}{m - \ln\left(\frac{p_w}{A}\right)}$
  
  where $p_w = RH \times p_{ws}(T)$ is the actual vapor pressure, $A$, $m$, and $T_n$ are empirical constants.
- **Humidity Ratio ($W$):** The mass of water vapor per unit mass of dry air:
  
  $W = 0.62198 \frac{p_w}{P - p_w}$
  
  where $P$ is the total atmospheric pressure.
- **Enthalpy ($h$):** The total heat content of moist air per unit mass of dry air:
  
  $h = c_p T + W (L_v + c_p T)$
  
  where $c_p$ is the specific heat of dry air, $L_v$ is the latent heat of vaporization, and $T$ is the dry bulb temperature.

## Usage
To use the `PSYCHROMETRICS` function in Excel, enter it as a formula in a cell, specifying the calculation type and required parameters:

```excel
=PSYCHROMETRICS(calc_type, dry_bulb_C, relative_humidity, [pressure_Pa])
```

## Arguments
| Argument           | Type   | Required | Description                                                      | Example         |
|--------------------|--------|----------|------------------------------------------------------------------|-----------------|
| calc_type          | string | Yes      | Calculation type: "wetbulb", "dewpoint", "humidityratio", "enthalpy" | "wetbulb"      |
| dry_bulb_C         | float  | Yes      | Dry bulb temperature in Celsius                                  | 25              |
| relative_humidity  | float  | Yes      | Relative humidity (0-100)                                        | 60              |
| pressure_Pa        | float  | No       | Atmospheric pressure in Pascals (default: 101325)                | 101325          |

## Returns
| Return Value | Type   | Description                                  | Example         |
|--------------|--------|----------------------------------------------|-----------------|
| Result       | float  | The result of the requested psychrometric calculation. | 18.23           |
| Error        | string | Error message if input is invalid or out-of-range.    | "Invalid input" |

## Limitations
- Only the calculation types listed above are supported.

## Benefits
- Useful for HVAC, weather, and process engineering tasks.
- Eliminates manual lookup of psychrometric charts.

## Examples
### Wet Bulb Temperature
Calculate the wet bulb temperature for $25\,\mathrm{^\circ C}$ and 60% RH:
```excel
=PSYCHROMETRICS("wetbulb", 25, 60)
```
### Dew Point
Calculate the dew point for $20\,\mathrm{^\circ C}$ and 80% RH:
```excel
=PSYCHROMETRICS("dewpoint", 20, 80)
```
### Humidity Ratio
Calculate the humidity ratio for $25\,\mathrm{^\circ C}$ and 40% RH:
```excel
=PSYCHROMETRICS("humidityratio", 25, 40)
```
### Enthalpy
Calculate the enthalpy for $18\,\mathrm{^\circ C}$ and 90% RH:
```excel
=PSYCHROMETRICS("enthalpy", 18, 90)
```

In [20]:
import math

def psychrometrics(calc_type, dry_bulb_C, relative_humidity, pressure_Pa=101325):
    """
    Perform core psychrometric calculations: wet bulb, dew point, humidity ratio, enthalpy

    Args:
        calc_type (str): Calculation type: 'wetbulb', 'dewpoint', 'humidityratio', 'enthalpy'
        dry_bulb_C (float): Dry bulb temperature in Celsius
        relative_humidity (float): Relative humidity (0-100)
        pressure_Pa (float, optional): Atmospheric pressure in Pascals (default: 101325)

    Returns:
        float or str: Result of the requested calculation, or error string if invalid
    """
    # Validate and convert inputs
    try:
        Tdb = float(dry_bulb_C)
        RH_input = float(relative_humidity)
        P = float(pressure_Pa)
    except Exception:
        return "Invalid input: could not convert arguments to float."

    if not (0 <= RH_input <= 100):
        return "Invalid input: relative_humidity must be between 0 and 100."
    RH = RH_input / 100.0

    if not isinstance(calc_type, str):
        return "Invalid input: unknown calculation type."
    calc_type_lower = calc_type.lower()

    # Constants
    A = 6.1121  # hPa
    m = 17.368
    Tn = 238.88  # deg C
    Cp = 1005  # J/kg.K
    Lv = 2501000  # J/kg
    Mw = 18.01528  # g/mol
    Md = 28.9644  # g/mol
    R = 8.31447  # J/(mol·K)

    # Helper: saturation vapor pressure (hPa)
    def pws(T):
        return A * math.exp((m * T) / (T + Tn))

    # Helper: actual vapor pressure (hPa)
    def pw(T, RH):
        return RH * pws(T)

    # Helper: humidity ratio (kg/kg)
    def humidity_ratio(T, RH, P):
        Pw = pw(T, RH) * 100  # Pa
        return 0.62198 * Pw / (P - Pw)

    # Helper: dew point (C)
    def dew_point(T, RH):
        Pw = pw(T, RH)
        lnPw = math.log(Pw / A)
        return (Tn * lnPw) / (m - lnPw)

    # Helper: enthalpy (kJ/kg)
    def enthalpy(T, RH, P):
        W = humidity_ratio(T, RH, P)
        return Cp * T + W * (Lv + Cp * T)  # J/kg

    # Helper: wet bulb (C) via iterative solution
    def wet_bulb(T, RH, P):
        # Approximate by iteration (Newton-Raphson)
        Twb = T
        for _ in range(20):
            Pw = pw(T, RH) * 100  # Pa
            W = 0.62198 * Pw / (P - Pw)
            # Guess Twb, compute saturation pressure at Twb
            Pws_wb = pws(Twb) * 100  # Pa
            W_wb = 0.62198 * Pws_wb / (P - Pws_wb)
            h_wb = Cp * Twb + W_wb * (Lv + Cp * Twb)
            h_air = Cp * T + W * (Lv + Cp * T)
            delta = h_air - h_wb
            if abs(delta) < 0.01:
                break
            Twb += delta / (Cp + (Lv + Cp * Twb) * 0.0001)
        return Twb

    if calc_type_lower == "wetbulb":
        return round(wet_bulb(Tdb, RH, P), 2)
    elif calc_type_lower == "dewpoint":
        return round(dew_point(Tdb, RH), 2)
    elif calc_type_lower == "humidityratio":
        return round(humidity_ratio(Tdb, RH, P), 6)
    elif calc_type_lower == "enthalpy":
        return round(enthalpy(Tdb, RH, P) / 1000, 2)  # kJ/kg
    else:
        return "Invalid input: unknown calculation type."
    # Fallback return for safety
    return "Invalid input: unknown error."

In [21]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_demo_wetbulb():
    result = psychrometrics(calc_type="wetbulb", dry_bulb_C=25, relative_humidity=60)
    assert isinstance(result, float), \
        f"Failed: demo_wetbulb - Calculate wet bulb temperature for 25°C, 60% RH. - Got: {result}"

def test_demo_dewpoint():
    result = psychrometrics(calc_type="dewpoint", dry_bulb_C=20, relative_humidity=80)
    assert isinstance(result, float), \
        f"Failed: demo_dewpoint - Calculate dew point for 20°C, 80% RH. - Got: {result}"

def test_demo_humidityratio():
    result = psychrometrics(calc_type="humidityratio", dry_bulb_C=25, relative_humidity=40)
    assert isinstance(result, float), \
        f"Failed: demo_humidityratio - Calculate humidity ratio for 25°C, 40% RH. - Got: {result}"

def test_demo_enthalpy():
    result = psychrometrics(calc_type="enthalpy", dry_bulb_C=18, relative_humidity=90)
    assert isinstance(result, float), \
        f"Failed: demo_enthalpy - Calculate enthalpy for 18°C, 90% RH. - Got: {result}"

def test_invalid_rh():
    result = psychrometrics(calc_type="wetbulb", dry_bulb_C=25, relative_humidity=120)
    assert result == "Invalid input: relative_humidity must be between 0 and 100.", \
        f"Failed: invalid_rh - Invalid RH (over 100%) should return error message. - Got: {result}"

def test_invalid_type():
    result = psychrometrics(calc_type="foobar", dry_bulb_C=25, relative_humidity=60)
    assert result == "Invalid input: unknown calculation type.", \
        f"Failed: invalid_type - Invalid calculation type should return error message. - Got: {result}"

ipytest.run()

Note: you may need to restart the kernel to use updated packages.
......                                                                                       [100%]
6 passed in 0.02s


<ExitCode.OK: 0>

In [22]:
# Gradio demo for psychrometrics function
import gradio as gr

# Gradio formatted examples for the demo
examples = [
    ["wetbulb", 25, 60],  # demo_wetbulb
    ["dewpoint", 20, 80],  # demo_dewpoint
    ["humidityratio", 25, 40],  # demo_humidityratio
    ["enthalpy", 18, 90],  # demo_enthalpy
]

demo = gr.Interface(
    fn=psychrometrics, # Use function directly without wrapper
    inputs=[
        gr.Dropdown(["wetbulb", "dewpoint", "humidityratio", "enthalpy"], label="Calculation Type"),
        gr.Number(label="Dry Bulb Temperature (°C)", value=25),
        gr.Number(label="Relative Humidity (%)", value=60),
        gr.Number(label="Pressure (Pa)", value=101325),
    ],
    outputs=gr.Number(label="Result"),
    examples=examples,
    description="Calculate wet bulb, dew point, humidity ratio, or enthalpy using the psychrometrics function.",
    flagging_mode="never",
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
